# Week 9 - Dimensionality reduction

In this lab, you'll get a chance to experiment with library implementations of the dimensionality reduction techniques discussed in lecture.

You will not be implementing the algorithms yourself. Instead you will be expected to use the library methods by reading the online documentation. This is intended to be practical experience - finding solutions based on existing libraries is usually much more practical than implementing a solution from scratch!

### Installation notes

To run this notebook you will need to install several packages:

    conda install pandas
    conda install scikit-learn
    conda install -c conda-forge umap-learn
    conda install -c conda-forge altair vega

In [ ]:
%matplotlib inline

In [ ]:
import pandas as pd
import gzip
import re
from urllib.request import urlretrieve
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE, MDS
from umap import UMAP

In [ ]:
import altair

In [ ]:
import matplotlib.pyplot as plt
# Set default figure size to a larger size
plt.rcParams['figure.figsize'] = [10, 10]

The following code will download the raw [FlyAtlas](http://flyatlas.org/atlas.cgi) data from [NCBI's Gene Expression Omnibus](https://www.ncbi.nlm.nih.gov/geo/). It may take a few moments to download.

In [ ]:
urlretrieve("ftp://ftp.ncbi.nlm.nih.gov/geo/series/GSE7nnn/GSE7763/matrix/GSE7763_series_matrix.txt.gz", filename="flydata.txt.gz")

The following two cells will open the compressed file you downloaded from GEO into a `pandas` data frame, then reopen the file and parse out the sample title line in order to use this as the column names for the data frame.

In [ ]:
with gzip.open("flydata.txt.gz") as handle:
    expression = pd.read_csv(handle, sep="\t", comment="!", index_col=0)

In [ ]:
with gzip.open("flydata.txt.gz") as handle:
    for line in handle:
        line = line.decode("utf-8")
        if line.startswith("!Sample_title"):
            header = [x.strip('"') for x in line.split("\t")[1:]]
            expression.columns = header
            break

Output the data frame and see what we have. It should have microarray probe ID's as row names (these can be mapped to gene names but we will skip that for today) and sample names as column names.

The data frame has 18952 rows (measurements) and 136 columns (samples) so it is certainly high dimensional.

In [ ]:
expression.head()

These 136 columns represent 4 replicates each from 34 different tissue types.

The following code snippet removes the replicate name from each sample, so we can use these labels as categories for plotting later.

In [ ]:
sample_categories = [re.match('(.+?)(( biological)? rep\d+)', c).group(1)
                     for c in expression.columns]

In [ ]:
sample_categories

The following function will render a two-dimensional scatterplot which is coloured by the list of categories. We will use it for PCA, MDS, tSNE, and UMAP visualizations.

The first function provided uses the Altair plotting library, which is interactive, allowing us to mouseover the points. To use this, you must install Altair as described in the first cell.

If you have trouble with Altair, you can use the second function below instead, which only requires matplotlib.

In [ ]:
# Here is a plotting function that uses Altair
# You can interact with the plot by mousing over
# the data points
def plot_two_dimensions(data, categories):
    df = pd.DataFrame(data)
    df.columns = ['Dim{}'.format(n) for n in range(1,data.shape[1]+1)]
    df['Category'] = sample_categories
    df['Sample'] = expression.columns
    chart = altair.Chart(df).mark_circle().\
                encode(x='Dim1',y='Dim2',color='Category',tooltip='Sample')
    return chart

In [ ]:
# Here is a plotting function that uses just matplotlib
# Use this if you have trouble with Altair
def plot_two_dimensions_mpl(data, categories):
    categories = pd.Series(categories)
    fig,ax = plt.subplots()
    for category in categories.unique():
        ax.scatter(data[categories==category, 0], 
                    data[categories==category, 1],
                    label=category)
    # Place the legend outside the plot, at x=1.05
    # (where the plot runs from 0 to 1)
    plt.legend(loc=(1.05,0))

The following code performs PCA on the dataset. The `expression.values` extracts the values in the data frame as a matrix. The `.T` takes the transpose of the matrix (swaps rows and columns).

In [ ]:
pca = PCA(n_components=2)
expression_pca = pca.fit(expression.values.T).transform(expression.values.T)
plot_two_dimensions(expression_pca, sample_categories)

This code prints out the variance explained by component.

In [ ]:
print(pca.explained_variance_ratio_)

This code makes a plot of the explained variance by component, like we saw on one of the lecture slides.

In [ ]:
plt.plot([x + 1 for x in range(len(pca.explained_variance_ratio_))], pca.explained_variance_ratio_, 'o-')
plt.xlabel("principal component")
plt.ylabel("variance explained")

**Exercise:**

Re-run the PCA with a higher number of dimensions and see how the plot of variance explained changes. How many components do you think are worth keeping if you were going to do this analysis?

**Exercise:**

Try creating an MDS plot using `MDS()`, which we imported from `sklearn.manifold`. All scikit-learn models use a consistent syntax, so the syntax is extremely similar to that for `PCA()`.

Examine the documentation either online or just using `help(MDS)` in the notebook.

In [ ]:
### YOUR CODE HERE

**Exercise:**

Try creating a tSNE plot using `TSNE()`, which we imported from `sklearn.manifold`. All scikit-learn models use a consistent syntax, so the syntax is extremely similar to that for `PCA()`.

Examine the documentation either online or just using `help(TSNE)` in the notebook.

`TSNE()` takes several parameters: the most important is `perplexity`. Lower values of perplexity try hard to preserve local structure at the cost of global structure, and vice versa. From the documentation, what is the default value of `perplexity`? What happens if you redo your plot with it set to a much lower or much higher value?

In [ ]:
### YOUR CODE HERE

**Exercise:**

Try creating a UMAP plot using `UMAP()`, which we imported from the `umap` library. `umap` is not part of scikit-learn, but it deliberately uses a similar syntax.

Examine the documentation either online or just using `help(UMAP)` in the notebook.

Look at the available parameters in the documentation. Try varying `n_neighbours` (which has a conceptual similarity to tSNE's `perplexity`) and `min_dist`.

In [ ]:
### YOUR CODE HERE